### Imports

In [1]:
import pandas as pd
import numpy as np

# import NeuralNet class
from somlib import NeuralNet

# Train neural network
load data

In [2]:
train_data = pd.read_csv("data/train.csv")
valid_data = pd.read_csv("data/test.csv")

In [3]:
train_data.head(3)

,ACC Macro,Bangdiwala B,Bennett S,Conditional Entropy,Cross Entropy,F1 Micro,FNR Micro,FPR Micro,Gwet AC1,Hamming Loss,...,Reference Entropy,Response Entropy,Standard Error,TNR Micro,TPR Micro,mark_1,mark_2,mark_3,mark_4,mark_5
0,0.997977,0.997977,0.995955,0.021018,0.002921,0.997977,0.002023,0.002023,0.997973,0.002023,...,-0.000000,0.021018,0.000025,0.997977,0.997977,1,0,0,0,0
1,0.998312,0.998309,0.996624,0.002434,0.027589,0.998312,0.001688,0.001688,0.998306,0.001688,...,0.026095,0.010551,0.000023,0.998312,0.998312,0,0,0,0,1
2,0.892172,0.878774,0.784344,0.487271,0.398881,0.892172,0.107828,0.107828,0.865341,0.107828,...,0.324197,0.646727,0.000175,0.892172,0.892172,0,0,0,1,0


Define architecture. 

Architecture of neural network defining by `dict` object. 

Every `key` of this `dict` is name of layer, e.g. "input", "layer_1", "first layer", ect. This names choose by user. For each layer (`key`) `value` is another `dict`, vith keys:
* `type`: layer type, required key; now available: 
 * `"fully_connected"` - for fully-connected layer;
 * `"convolution"` - for convolution layer;
 * `"max_pool"` - for max pooling layer;
 * `"flatten"` - for reshape n-demetion tensors (for example: output of convolution layer) into vector (one-dimention object). For batch of objects reshape all objects in batch, but save first dimention $(10 \times 5 \times 3 \times 2) \rightarrow (10 \times 30)$
 * `"out"` - for last fully-connected layer (e.g. output of model)
* "`activation"` - non-lineary function for all layers, <u>except</u> **max pooling** and **flatten**, required key; now available: 
 * `"sigmoid"`:
 $$ h_ \theta (x) =  \frac{\mathrm{1} }{\mathrm{1} + e^- \theta^Tx }  $$
 * `"tanh"`:
 $$ tanh(x) = \frac{e^{2x} - 1}{e^{2x} + 1}$$
 * `"relu"`:
 $$ReLU(x) = max(x, 0)$$
 * `"softmax"`:
 $$\sigma (x)_{i} = \frac{e^{x_{i}}}{\sum^{K}_{k=1}{e^{x_{k}}}}$$
* specific keys for **convolution layer**:
 * `"filtres"`: number of filters in convolution layer, dtype: `int`;
 * `"kernel"` : shape of filter (convolution kernel), this key recive list of 2 int for "width" and "heigh" of convolution kernel, dtype: `list`;
 * `"stride"` : stride along "width" and "heigh" for convolution operation, this key recive list of 2 int for "width" and "heigh" stride, dtype: `list`;
 * `"pad"` : padding input tensor with zeros along "width" and "heigh", this key recive list of 2 int for "width" and "heigh" stride, dtype: `list`;
* specific keys for **max pooling layer**:
 * `"kernel"` : shape of max pooling mask, this key recive list of 2 int for "width" and "heigh" of max pooling mask, dtype: `list`;
 * `"stride"` : stride along "width" and "heigh" for max pooling, this key recive list of 2 int for "width" and "heigh" stride, dtype: `list`;
* specific keys for **fully-connected layer**:
 * `"neurons"`: number of hidden units (neurons) into layer, dtype: `int`
* **flatten layer** has no parameters.

In [4]:
architecture = {
    "input": {"type": "fully_conneted", "neurons": 31, "activation": "sigmoid"},
    "hidden": {"type": "fully_conneted", "neurons": 18, "activation": "sigmoid"},
    "out": {"type": "out", "neurons": 5, "activation": "sigmoid"},
}

Define basic settings of model

Settings of model is `dict` with specific keys, from this settings class `NeuralNetwork` build tensorflow computation graph. Settings have number of `"keys"`:
* `"outs"`: number of output neurons (must match with `"neurons"` in `"out"` layer), dtype: `int`;
* `"batch_size"`: batch size for trainig and runnig model, if data amount smallest than 5000 reccomend use all data in one batch, dtype: `int`;
* `"architecture"`: dict with architecture (see above), dtype: `dict`;
* `"inputs"`: shape of input object (ont training example), in case of fulle-connected network must bi `list` with one value, dtype: `list`

In [5]:
batch_size = len(train_data)
inputs_len = len(train_data.columns) - 5
settings = {
    "outs": 5,
    "batch_size": batch_size,
    "architecture": architecture,
    "inputs": [inputs_len],
}

Build computation graph for neural network

In [6]:
nn = NeuralNet(settings, verbose=True)

=====>Neural net info<===== 

Settings: 
+------------+-------+
|  Setting   | Value |
+------------+-------+
|    outs    |   5   |
| batch_size |  240  |
|   inputs   |  [23] |
+------------+-------+

tf version:  2.2.0 

Complex:
        [parameters]x[batch size]
        1415x240

Architecture:
+--------+----------------+----------------+------------+
|  Name  |      Type      | Neurons number | Activation |
+--------+----------------+----------------+------------+
| input  |     input      |      [23]      |     -      |
| input  | fully_conneted |       31       |  sigmoid   |
| hidden | fully_conneted |       18       |  sigmoid   |
|  out   |      out       |       5        |  sigmoid   |
+--------+----------------+----------------+------------+




Train neural net

In [7]:
nn.fit_lm(
    x_train=train_data.values[:, :-5],
    y_train=train_data.values[:, -5:],
    x_valid=valid_data.values[:, :-5],
    y_valid=valid_data.values[:, -5:],
    mu_init=5.0,
    min_error=2.083e-4,
    max_steps=10,
    mu_multiply=10,
    mu_divide=10,
    m_into_epoch=5,
    verbose=True,
    random_batches=True,
    plot_widget=True
)

LM step: 1, train mse: 5.03e-01 test mse: 5.04e-01
LM step: 2, train mse: 4.98e-01 test mse: 4.99e-01
LM step: 3, train mse: 4.59e-01 test mse: 4.58e-01
LM step: 4, train mse: 3.49e-01 test mse: 3.41e-01
LM step: 5, train mse: 3.25e-01 test mse: 3.16e-01
LM step: 6, train mse: 3.19e-01 test mse: 3.09e-01
LM step: 7, train mse: 3.19e-01 test mse: 3.09e-01
LM step: 8, train mse: 3.21e-01 test mse: 3.11e-01
LM step: 9, train mse: 3.19e-01 test mse: 3.10e-01
LM step: 10, train mse: 3.19e-01 test mse: 3.10e-01
LevMarq ended on: 10,	final loss: 3.19e-01

